# AVI API Playground

This is for first steps... of AVI by using Jupyter

https://avinetworks.com/docs/latest/api-guide/overview.html 


https://github.com/avinetworks/sdk/blob/master/python/avi/sdk/README.md

[1.- Initial Login](#1--initial-login)
[2.- Read Operations](#import-libraries)

## 1.- Initial Login

In this step bla, bla, bla


### Import Libraries

In [22]:
from avi.sdk.avi_api import ApiSession
import datetime, time
from requests.packages import urllib3
urllib3.disable_warnings()

### Import Environment Session Variables

In [23]:
from envs.controller_info import session_params as session_env

In [24]:
# Establish a first session
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Update headers and api version imported from demo env file with controller version (ensure actual API Version is uses in subsequent requests
session_env['headers']['X-Avi-Version'] = api.remote_api_version['Version']
session_env['api_version'] = api.remote_api_version['Version']

# Create a new session with received AVI API Version
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )

### Display Session ID to Verify AVI Controller Session Establishment

In [25]:
print('Successful connection to ' + session_env['name'] + '. Session ID:' + api.session_id)

Successful connection to avicontroller. Session ID:6mo8t0rxrui350rexjx18kxcpawh0g2q


## 2.- Read Operations

Get is a common method to read information.
You can get information of your current setup easily invoking the corresponding API

AVI has dozens of APIs... 
   - Use https://controller/swagger to understand different API
   - You can easily infere the API when navigating through the GUI just looking at the address bar used in the GUI 

   Example if the GUI shows https://192.168.1.15/#!/admin/applications/virtualservice that means you are in the **admin** tenant and you are using the **virtualservice** API

   In the previous step we created an AVI API session object that we called *api*. 
   Let's invoke the get method over our api object to inspect the output

### 2.1 API Object **Get** Method

Import some required Libraries first

In [26]:
import json

Now create a new object named vs with will contain the output of the get method using the virtualservice API path

In [27]:
vs = api.get('virtualservice')

The request produces a json formatted output. You can change it by using 

In [28]:
json.loads(vs.text)

{'count': 2,
 'results': [{'_last_modified': '1720707512296814',
   'active_standby_se_tag': 'ACTIVE_STANDBY_SE_1',
   'advertise_down_vs': False,
   'allow_invalid_client_cert': False,
   'analytics_policy': {'all_headers': True,
    'client_insights': 'NO_INSIGHTS',
    'full_client_logs': {'duration': 0, 'enabled': True, 'throttle': 10},
    'learning_log_policy': {'enabled': False},
    'metrics_realtime_update': {'duration': 0, 'enabled': True},
    'significant_log_throttle': 10,
    'udf_log_throttle': 10},
   'analytics_profile_ref': 'https://192.168.1.15/api/analyticsprofile/analyticsprofile-768ef0ed-a2ac-427c-92c7-8715ce805374',
   'application_profile_ref': 'https://192.168.1.15/api/applicationprofile/applicationprofile-a70f9a51-201c-428a-ac55-8d70e840650f',
   'bulk_sync_kvcache': False,
   'close_client_conn_on_config_update': False,
   'cloud_ref': 'https://192.168.1.15/api/cloud/cloud-baf1f7f6-18ff-46cb-a134-6154d9af52a1',
   'cloud_type': 'CLOUD_NONE',
   'content_rewri

As you can tell, above output can be hard to handle and read. There are some parameters you can use to narrow down the response.

- **path**: takes relative path to the AVI api 
- **tenant**: overrides the tenant used during session creation
- **tenant_uuid**: overrides the tenant or tenant_uuid during session creation  
- **timeout**: timeout for API calls; Default value is 60 seconds 
- **api_version**: overrides x-avi-header in request header during session creation get method takes relative path to service and kwargs as per Session class get method returns session's response object
- **params**: dictionary of key value pairs to be sent as query parameters


| Filter                                               | Description                                                                   | Example                                                                                                                                                                                                    | Returns                                                                                                                                                         |
| ---------------------------------------------------- | ----------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| \<field\>=                                             | Filter objects with specific fields matching given value                      | /api/pool/?fail_action.type=FAIL_ACTION_CLOSE_CONN                                                                                                                                                         | Returns list of pools whose failure action is configured as FAIL_ACTION_CLOSE_CONN                                                                              |
|                                                      |\<field\> can be a nested field using . as a separator | /api/pool/?analytics_policy.enable_realtime_metrics=false                     | Returns list of pools with real-time metrics enabled                                                                                                                                                       |
| \<field\>.contains=                                    | Filter objects where field \<field\> contains the given string                  | /api/pool/?name.contains=-app1-                                                                                                                                                                            | Returns list of pools whose name contains the string "-app1-"                                                                                                   |
| \<field\>.icontains=                                   | ("contains" is case-sensitive, "icontains" is case-insensitive)               |
| [name.in](http://name.in)                            | Filter objects where name or uuid exactly matches any of the values in a list | /api/pool/?name.in=pool-1,pool-2,pool-3                                                                                                                                                                    | Returns list with pool-1, pool-2 and pool-3                                                                                                                     |
| [uuid.in](http://uuid.in)                            |
| search                                               | Filter objects which contain the search string in any field’s value           | /api/pool/?search=LEAST_CONN                                                                                                                                                                               | Returns list of pools containing "LEAST_CONN" in any field value. To search for an exact match on an entire field’s value, enclose the search string in quotes. |
| isearch                                              | /api/pool/?search="LB_ALGORITHM_LEAST_CONNECTIONS"                            | ("search" is case-sensitive, "isearch" is case-insensitive)                                                                                                                                                |
| search=()                                            | Filter objects by searching for a field name and value                        | /api/pool/?search=(addr,10.1.1.1)                                                                                                                                                                          | Returns list of pools containing the string "10.1.1.1" in the field "addr" (even if the field is a nested field as in this case).                               |
| isearch=()                                           | /api/pool/?search=(addr, "10.1.1.1")                                          | To search for an exact match on an entire field’s value, enclose the search string in quotes. Leaving the value part empty matches any value (i.e. searches for the presence of the specified field name). |
|                                                      | /api/virtualservice/?search=(dns_policies,)                                   | ("search" is case-sensitive, "isearch" is case-insensitive)                                                                                                                                                |
| refers_to                                            | Filter objects which have a reference to the specified object                 | /api/pool/?refers_to=healthmonitor:healthmonitor-be7880b9-2c72-4abd-9fd3-8243517e8163                                                                                                                      | Returns list of pools that refer to the health monitor with this UUID                                                                                           |
| referred_by                                          | Filter objects which are referred to by the specified object                  | /api/pool/?referred_by=virtualservice:virtualservice-7a292eb3-ad23-4900-a269-c529db4133f8                                                                                                                  | Returns list of pools that are referred to by the virtual service with this UUID                                                                                |
| exclude                                              | Inverts the filtering logic of the specified query                            | /api/pool/?name.icontains=-web-&refers_to=healthmonitor:healthmonitor-be7880b9-2c72-4abd-9fd3-8243517e8163&exclude=name.icontains                                                                          | Returns list of pools that refer to the health monitor with this UUID where the Pool name does not contain the string "-web-"                                   |
| skip_default                                         | Excludes fields from the returned object that are set to the default value    | /api/pool/?name=app1-pool&skip_default=true                                                                                                                                                                | Returns the specified object excluding any field whose value has not been changed from its default                                                              |
| include_name                                         | Appends the object name to each object reference - \<object\>_ref: \<url\>#\<name\> | /api/pool/?name=app1-pool&include_name=true                                                                                                                                                                | Returns the object with all \*_ref fields having the referenced object name appended                                                                            |

**Example 1**.- Get virtualservices with a name value equals to **vs-example-org1**

_Sample output:_

```json
{'count': 1,
 'results': [{'_last_modified': '1720707512296814',
   'active_standby_se_tag': 'ACTIVE_STANDBY_SE_1',
   'advertise_down_vs': False,
   'allow_invalid_client_cert': False,
   ... # (skipped)
   'close_client_conn_on_config_update': False,
   'cloud_ref': 'https://192.168.1.15/api/cloud/cloud-baf1f7f6-18ff-46cb-a134-6154d9af52a1',
   'cloud_type': 'CLOUD_NONE',
   'content_rewrite': {'rewritable_content_ref': 'https://192.168.1.15/api/stringgroup/stringgroup-caeef771-f17f-4099-936b-33023eb37300'},
   'delay_fairness': False,
   'east_west_placement': False,
   'weight': 1}]}
```

In [ ]:
query= { 
    "name": "vs-example-org1"
    }
vs = api.get('virtualservice', params=query)
json.loads(vs.text)

**Example 2**.- Get virtualservices with a name value equal to vs-example-org1 but only retrieve some fields in the response (enabled, vsvip_ref and type).

_Sample output_:

```json
{'count': 1,
 'results': [
    {'enabled': True,
   'name': 'vs-example-org1',
   'type': 'VS_TYPE_NORMAL',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c',
   'uuid': 'virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c',
   'vsvip_ref': 'https://192.168.1.15/api/vsvip/vsvip-188553aa-bbe1-4b6e-b2ed-7ed1d5f78293'}
  ]
}
```

In [ ]:
query= { 
    "name": "vs-example-org1", 
    "fields": "enabled,vsvip_ref,type"
    }
vs = api.get('virtualservice', params=query)
json.loads(vs.text)

**Example 3**.- Get virtualservices that contains the string **-Example-** in the field **name** (key insensitive) and only retrieve some fields in the response (name).

_Sample output_:

```json
{'count': 2,
 'results': [{'name': 'vs-example-org1',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c',
   'uuid': 'virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c'},
  {'name': 'vs-example-org1-new-name',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-ad04064c-f206-40c2-95b8-a69bd6d57646',
   'uuid': 'virtualservice-ad04064c-f206-40c2-95b8-a69bd6d57646'}]}
```

In [90]:
query= { 
    "name.icontains": "-Example-", 
    "fields": "name"
    }
vs = api.get('virtualservice', params=query)
json.loads(vs.text)

{'count': 2,
 'results': [{'name': 'vs-example-org1',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c',
   'uuid': 'virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c'},
  {'name': 'vs-example-org1-new-name',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-ad04064c-f206-40c2-95b8-a69bd6d57646',
   'uuid': 'virtualservice-ad04064c-f206-40c2-95b8-a69bd6d57646'}]}

**Example 4**.- Using two queries obtain the virtualservices sharing the same service engine group with a given virtualservice named **vs-example-org1**.
    
  - 1st query get se_group_ref for a given virtualservice name  

_Sample output_first_query:_

```json
{'count': 1,
 'results': [{'name': 'vs-example-org1',
   'se_group_ref': 'https://192.168.1.15/api/serviceenginegroup/serviceenginegroup-17a49de4-afad-4929-a8e1-297be6ab9d70#SOURCE-GROUP1',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c#vs-example-org1',
   'uuid': 'virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c'}]}
```
  - 2nd query after extracting the UUID (embedded in the se_group_ref retrieved field above) use the refers_to query parameter to get all virtualservice sharing same service engine group

_Sample output_second_query:_

```json
{'count': 2,
 'results': [{'name': 'vs-example-org1',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c',
   'uuid': 'virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c'},
  {'name': 'vs-example2-org1',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-7779e1cb-c7b8-4050-a266-1ab699dd4076',
   'uuid': 'virtualservice-7779e1cb-c7b8-4050-a266-1ab699dd4076'}]}
```


In [149]:
# 1st query retrieve se_group_ref appending the name to ease identification
query= {
    "name": "vs-example-org1", 
    "fields": "name,se_group_ref", # Without blank spaces
    "include_name": "true"
    }
vs = api.get('virtualservice', params=query)
json.loads(vs.text)

{'count': 1,
 'results': [{'name': 'vs-example-org1',
   'se_group_ref': 'https://192.168.1.15/api/serviceenginegroup/serviceenginegroup-17a49de4-afad-4929-a8e1-297be6ab9d70#SOURCE-GROUP1',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c#vs-example-org1',
   'uuid': 'virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c'}]}

Extract the se_group_ref (removing the trailing name delimited by #)

In [147]:
# Copy response text to the output variable
output=json.loads(vs.text)
# Access the first (0 index) of results and extract the se_group_ref field splitting the trailing name delimited by #. Store in se_group_ref_name variable
se_group_ref_name = output["results"][0]["se_group_ref"].split("#")[0]
# Extract the SE_GROUP_UUID from the se_group_ref (last part of URL)
se_group_uuid = se_group_ref_name.split("/")[5]
print("The extracted Service Engine Group UUID is "+se_group_uuid)

The extracted Service Engine Group UUID is serviceenginegroup-17a49de4-afad-4929-a8e1-297be6ab9d70


In [151]:
# 2st query retrieve all virtual services that refer to the service Engine Group UUID by that se_group_ref
query= {
    "fields": "name", # Without blank spaces
    "refers_to": f"serviceenginegroup:{se_group_uuid}"
    }
vs = api.get('virtualservice', params=query)
json.loads(vs.text)

{'count': 2,
 'results': [{'name': 'vs-example-org1',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c',
   'uuid': 'virtualservice-f851f486-66d3-48ed-bfaa-d749c477f91c'},
  {'name': 'vs-example2-org1',
   'url': 'https://192.168.1.15/api/virtualservice/virtualservice-7779e1cb-c7b8-4050-a266-1ab699dd4076',
   'uuid': 'virtualservice-7779e1cb-c7b8-4050-a266-1ab699dd4076'}]}